<a href="https://colab.research.google.com/github/JakubPac/kurs_ml/blob/main/apriori_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from keras.datasets import mnist
np.set_printoptions(precision = 4, suppress = True, edgeitems = 5, linewidth = 200)

In [2]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv))

--2025-09-01 13:19:19--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.180.207, 192.178.218.207, 172.253.115.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.180.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2166953 (2.1M) [application/octet-stream]
Saving to: ‘products.csv’

products.csv        100%[===================>]   2.07M  3.26MB/s    in 0.6s    

2025-09-01 13:19:20 (3.26 MB/s) - ‘products.csv’ saved [2166953/2166953]

--2025-09-01 13:19:20--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.180.207, 192.178.218.207, 172.253.115.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.180.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24680147 (24M) [application/octet-stre

In [4]:
products = pd.read_csv('products.csv', usecols = ['product_id', 'product_name'])
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [5]:
orders = pd.read_csv('orders.csv', usecols = ['order_id', 'product_id'])
orders.head()

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633


Przygotowanie danych


In [6]:
data = pd.merge(orders, products, how = 'inner', on = 'product_id', sort = 'True')
data.sort_values(by = 'order_id')
data.head()

,order_id,product_id,product_name
0,6695,1,Chocolate Sandwich Cookies
1,48361,1,Chocolate Sandwich Cookies
2,63770,1,Chocolate Sandwich Cookies
3,75339,1,Chocolate Sandwich Cookies
4,240996,1,Chocolate Sandwich Cookies


In [7]:
data.describe()

,order_id,product_id
count,1.384617e+06,1.384617e+06
mean,1.706298e+06,2.555624e+04
std,9.897326e+05,1.412127e+04
min,1.000000e+00,1.000000e+00
25%,8.433700e+05,1.338000e+04
50%,1.701880e+06,2.529800e+04
75%,2.568023e+06,3.794000e+04
max,3.421070e+06,4.968800e+04


In [8]:
data['product_name'].value_counts()

,count
product_name,
Banana,18726
Bag of Organic Bananas,15480
Organic Strawberries,10894
Organic Baby Spinach,9784
Large Lemon,8135
...,...
Vegetable & Chicken Dumpling,1
"Baby Food Blueberry, Parsnip & Buckwheat Stage 2",1
Silicone Baking Mat,1


In [11]:
data['order_id'].nunique()

131209

In [12]:
transactions = data.groupby(by = 'order_id')['product_name'].apply(lambda name: ','.join(name))
transactions

,product_name
order_id,
1,"Organic Celery Hearts,Organic 4% Milk Fat Whol..."
36,"Spring Water,Prosciutto, Americano,Grated Peco..."
38,"Organic Raw Unfiltered Apple Cider Vinegar,She..."
96,"Roasted Turkey,Organic Whole Strawberries,Orga..."
98,"Organic Whole Grassmilk Milk,Garbanzo Beans,Ge..."
...,...
3421049,"Organic Grated Ginger,Organic Whole Grain Whea..."
3421056,"Total Plain Greek Strained Yogurt,Homestyle Cl..."
3421058,"Extralean Ground Turkey Breast,Club Soda Lower..."


In [13]:
transactions = transactions.str.split(',')
transactions

,product_name
order_id,
1,"[Organic Celery Hearts, Organic 4% Milk Fat Wh..."
36,"[Spring Water, Prosciutto, Americano, Grated ..."
38,"[Organic Raw Unfiltered Apple Cider Vinegar, S..."
96,"[Roasted Turkey, Organic Whole Strawberries, O..."
98,"[Organic Whole Grassmilk Milk, Garbanzo Beans,..."
...,...
3421049,"[Organic Grated Ginger, Organic Whole Grain Wh..."
3421056,"[Total Plain Greek Strained Yogurt, Homestyle ..."
3421058,"[Extralean Ground Turkey Breast, Club Soda Low..."


Kodowanie transakci

In [14]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()
encoder.fit(transactions)
transactions_encoded = encoder.transform(transactions, sparse = True)
transactions_encoded

<Compressed Sparse Row sparse matrix of dtype 'bool'
	with 1442410 stored elements and shape (131209, 40434)>

In [15]:
transactions_encoded_df = pd.DataFrame(transactions_encoded.toarray(), columns = encoder.columns_)
transactions_encoded_df

,,Apricot & Banana Stage 2 Baby Food,Broad Spectrum SPF 30,Instant,Livermore Valley,Low Sodium Marinara,Premium,Vetiver scent,Whole,#2,...,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water,Lightly Seasoned with Rosemary and Roasted Garlic Family Size Herb Chicken Tortellini
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131205,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131206,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131207,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Algorytm apriori

In [18]:
from mlxtend.frequent_patterns import apriori, association_rules

supports = apriori(transactions_encoded_df, min_support = 0.01, use_colnames = True)
supports = supports.sort_values(by = 'support', ascending = False)
supports.head(10)

,support,itemsets
8,0.143138,(Banana)
7,0.117980,(Bag of Organic Bananas)
76,0.083028,(Organic Strawberries)
41,0.074568,(Organic Baby Spinach)
31,0.062000,(Large Lemon)
37,0.056467,(Organic Avocado)
61,0.055583,(Organic Hass Avocado)
100,0.049494,(Strawberries)
33,0.045980,(Limes)
69,0.042268,(Organic Raspberries)


In [22]:
rules = association_rules(supports, metric = 'confidence', min_threshold = 0)
rules = rules.iloc[:,[0,1,4,5,6]]
rules = rules.sort_values(by = 'lift', ascending = False)
rules.head(15)

,antecedents,consequents,support,confidence,lift
27,(Clementines),( Bag),0.011295,0.523860,36.835557
26,( Bag),(Clementines),0.011295,0.794212,36.835557
22,(Large Lemon),(Limes),0.012156,0.196066,4.264159
23,(Limes),(Large Lemon),0.012156,0.264379,4.264159
19,(Organic Raspberries),(Organic Strawberries),0.012728,0.301118,3.626710
18,(Organic Strawberries),(Organic Raspberries),0.012728,0.153295,3.626710
31,(Organic Avocado),(Large Lemon),0.010281,0.182076,2.936692
30,(Large Lemon),(Organic Avocado),0.010281,0.165827,2.936692
3,(Organic Hass Avocado),(Bag of Organic Bananas),0.018444,0.331825,2.812560
2,(Bag of Organic Bananas),(Organic Hass Avocado),0.018444,0.156331,2.812560
